In [ ]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
import matplotlib.pyplot as plt

In [ ]:
tickers = ['PG', 'BEI.DE']
sec_prices = pd.DataFrame()
for t in tickers:
    sec_prices[t] = wb.DataReader(t, data_source='yahoo', start='2011-1-1')['Adj Close']

In [ ]:
# # In case of API slow responces 
# sec_prices = pd.DataFrame()
# sec_prices['PG'] = wb.DataReader('PG', data_source='yahoo', start='2011-1-1')['Adj Close']
# sec_prices['BEI.DE'] = wb.DataReader('BEI.DE', data_source='yahoo', start='2011-1-1')['Adj Close']

In [ ]:
# getting Log Return
sec_returns = np.log(sec_prices / sec_prices.shift(1))

## Variance and standard deviation as risk measures

Variance (dispersion): $s^2=\frac{\sum(x_i - \overline{x})^2}{N-1}$, where $x_i$ - elements of the sample, $\overline{x}$ - sample mean value, $N$ - the number of elements in the sample.

Standard deviation: $\sigma = \sqrt{s^2}$

Annualized variance: $s^2 * 250$

Annualized standard deviation: $\sqrt{s^2 * 250} = \sigma*\sqrt{250}$

Into the risks you have to include industry growth, revenue, profitability, regulatory environment.

In [ ]:
# Annualized avarage Log Return
sec_returns[['PG', 'BEI.DE']].mean() * 250

PG        0.098101
BEI.DE    0.094423
dtype: float64

In [ ]:
# Annuazlized standard deviation of Log Return
sec_returns[['PG', 'BEI.DE']].std() * 250 ** 0.5

PG        0.174198
BEI.DE    0.193958
dtype: float64

## Correlation between stocks

Covariance: $cov_{x,y}= \frac{\sum(x_i-\overline{x})(y_i-\overline{y})}{N-1}$, where $N$ - number of observations $(x_i, y_i)$. Also you may meet denotation $\rho_{x,y} := cov_{x,y}$.

One may perceive covariance as a measure of the variance of some sample where each observation consists of 2 (or more) variables.

<br/>

Correlation (Pearson's correlation coefficient): $r_{xy}= \frac{\sum(x_i-\overline{x})(y_i-\overline{y})}{\sqrt{\sum(x_i-\overline{x})^2\sum(y_i-\overline{y})^2}} = \frac{cov_{x,y}}{\sigma_x \sigma_y}$.

$r_{xy}\in[-1,1]$ and tell how strong is the relation between two variables. The relation, by itself, usually is not direct and not obvious. 

<br/>

Covariance matrix (for 3 variables 1 vs 2 vs 3):
$\begin{bmatrix}
cov_{1,1} & cov_{1,2} & cov_{1,3}\\
cov_{2,1} & cov_{2,2} & cov_{2,3}\\
cov_{3,1} & cov_{3,2} & cov_{3,3}
\end{bmatrix}$ =
$\begin{bmatrix}
\sigma_1^2 & cov_{1,2} & cov_{1,3}\\
cov_{2,1} & \sigma_2^2 & cov_{2,3}\\
cov_{3,1} & cov_{3,2} & \sigma_3^2
\end{bmatrix}$,

where symmetrical values are equal (e.g. $cov_{2,1}=cov_{1,2}$).

In [ ]:
# PG_var = sec_returns['PG'].var()
# PG_var_ann = sec_returns['PG'].var() * 250
# PG_var_ann

In [ ]:
# Annualized covariance matrix
sec_returns[['PG', 'BEI.DE']].cov() * 250

,PG,BEI.DE
PG,0.030345,0.009332
BEI.DE,0.009332,0.037620


In [ ]:
# Correlation between two stocks
sec_returns.corr()

,PG,BEI.DE
PG,1.000000,0.276399
BEI.DE,0.276399,1.000000


We consider `corr(returns)` as it focuses on the returns of the portfolio, when `corr(prices)` focuses just on stock price levels.

## Portfolio variance and volatility

Portfolio variance (2 stocks): $(\omega_1\sigma_1+ \omega_2\sigma_2)^2= \omega_1^2\sigma_1^2+ \omega_2^2\sigma_2^2+ 2\omega_1\sigma_1 \omega_2\sigma_2 \rho_{12}= \omega_1^2\sigma_1^2+ \omega_2^2\sigma_2^2+ 2\omega_1\omega_2*cov_{12}$,

where $\omega_1 + \omega_2 = 1$.

Equal weigthing scheme

In [ ]:
weights = np.array([0.5, 0.5])

### Portfolio variance

In case of numbers: $(ab)^2= a^2b^2$

In case of vectors and matrices: $(w*{Cov})^2= w^T*{Cov}*w= [w_1, w_2] * \begin{bmatrix}
\sigma_1^2 & cov_{1,2}\\
cov_{2,1} & \sigma_2^2
\end{bmatrix} *
\begin{bmatrix}
\omega_1\\
\omega_2
\end{bmatrix}$

In [ ]:
pfolio_var = np.dot(weights.T, np.dot(sec_returns.cov() * 250, weights))
pfolio_var

0.021657063038074756

### Portfolio volatility

 ${volatility}= \sqrt{varianace}$ 

In [ ]:
pfolio_vol = (np.dot(weights.T, np.dot(sec_returns.cov() * 250, weights))) ** 0.5
pfolio_vol

0.1471633889188298

## Systematic & non-systematic risks (idiosyncratic)

<u>Systemtaic risk</u> - is the uncertainty that is characteristic of the entire market.
Systematic risk is made of the day to day changes in stock prices and is
caused by events that affect all companies (e.g. recession of the economy, low customer spending, wars, forces of nature).

Systemtaic risks are considered to be un-diversifieble.

<u>Unsystematic risk</u> - is company-specific, industry-specific, risks that can be smoothed out through diversification. It can be eliminated by investing in non/low-correlated assets (e.g. automotive + construction + energy + technology). Investment traking indices (e.g. S&P 500) are well diversified. 

Unsystematic risks are considered to be diversifiable.


### Evaluation of diversifieble and undiversifieble risks of portfolio

`diversifiable_risk` = `annual_portfolio_variance` - `wighted_annual_variances_of_each_stock`

$2\omega_1\omega_2*cov_{12} = (\omega_1\sigma_1+ \omega_2\sigma_2)^2- (\omega_1^2\sigma_1^2+ \omega_2^2\sigma_2^2)$

<br/>

`undiversifiable_risk` = `wighted_annual_variances_of_each_stock` =  `annual_portfolio_variance` - `diversifiable_risk`

$(\omega_1^2\sigma_1^2+ \omega_2^2\sigma_2^2) = (\omega_1\sigma_1+ \omega_2\sigma_2)^2 - 2\omega_1\omega_2*cov_{12}$ 

In [ ]:
# weights of two assets
weights = np.array([0.5, 0.5])

In [ ]:
# Annualized variance
PG_var_ann = sec_returns['PG'].var() * 250
BEI_var_ann = sec_returns['BEI.DE'].var() * 250
print("Wighted annual variances of each stock: {} %" \
      .format(round((weights[0]**2 * PG_var_ann + weights[1]**2 * BEI_var_ann)*100, 3)))

Wighted annual variances of each stock: 1.699 %


In [ ]:
print("Portfolio variance: {} %".format(round(pfolio_var * 100, 3)))

Portfolio variance: 2.166 %


In [ ]:
# diversifiable risk
d_risk = pfolio_var - (weights[0]**2 * PG_var_ann) - (weights[1]**2 * BEI_var_ann)
print("Diversifiable risk: {} %".format(round(d_risk*100, 3)))

Diversifiable risk: 0.467 %


In [ ]:
# undiversifiable risk
ud_risk = pfolio_var - d_risk
print("Undiversifiable risk: {} %".format(round(ud_risk*100, 3)))

Undiversifiable risk: 1.699 %


## Fin